<a href="https://colab.research.google.com/github/alleyezonme1/mylabs1/blob/main/%D0%BB%D1%801.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!ls

2_5303101742206956494.fit  sample_data


In [ ]:
!pip install fitdecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 2.0 MB/s eta 0:00:00


In [ ]:
record = ['latitude', 'longitude', 'altitude', 'timestamp', 'heart_rate', 'cadence', 'speed', 'distance']

In [ ]:
#подключаем к блокноту пакеты
import fitdecode
import pandas as pd
import os
from datetime import datetime, timedelta
from typing import Dict, Union, Optional,Tuple

In [ ]:
#функция, которая извлекает данные о координатах и преобразует их в десятичные градусы
def get_fit_point_data(frame: fitdecode.records.FitDataMessage) -> Optional[Dict[str, Union[float, int, str, datetime]]]:
    data: Dict[str, Union[float, int, str, datetime]] = {}
    if not (frame.has_field('position_lat') and frame.has_field('position_long')):
        return None

    # привести значение координат к float
    else:
        lat_ = frame.get_value('position_lat')
        long_ = frame.get_value('position_long')
        if (lat_ is not None) and (long_ is not None):
            data['latitude'] = float(lat_) / ((2 ** 32) / 360)
            data['longitude'] = float(long_) / ((2 ** 32) / 360)
        else:
            return None  # Пропустить точки с отсутствующими координатами

    for field in record[2:]:
        if frame.has_field(field):
            data[field] = frame.get_value(field)

    return data

In [ ]:
#функция, которая извлекает данные о параметрах тренировки
def get_fit_other_data(col, frame: fitdecode.records.FitDataMessage) -> Optional[Dict[str, Union[float, int, str, datetime]]]:
  data: Dict[str, Union[float, int, str, datatime]] = {}

  for field in col:
    if frame.has_field(field):
      data[field] = frame.get_value(field)
  return data

In [ ]:
#чтение файла, вызов функций и запись dataframe
record_data = []
with fitdecode.FitReader('/content/2_5303101742206956494.fit') as fit_file:
  for frame in fit_file:
    if isinstance(frame, fitdecode.records.FitDataMessage):
      if frame.name == 'record':
        single_point_data = get_fit_point_data(frame)
        if single_point_data is not None:
          record_data.append(single_point_data)
record_df = pd.DataFrame(record_data, columns=record)
#record_df

/usr/local/lib/python3.10/dist-packages/fitdecode/reader.py:909: UserWarning: 'field "native_field_num" (idx #0) not found in message "field_description"' (local_mesg_num: 0; chunk_offset: 194); adding dummy dev data...
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/fitdecode/reader.py:909: UserWarning: 'field "native_field_num" (idx #0) not found in message "field_description"' (local_mesg_num: 0; chunk_offset: 236); adding dummy dev data...
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/fitdecode/reader.py:909: UserWarning: 'field "native_field_num" (idx #0) not found in message "field_description"' (local_mesg_num: 0; chunk_offset: 287); adding dummy dev data...
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/fitdecode/reader.py:909: UserWarning: 'field "native_field_num" (idx #0) not found in message "field_description"' (local_mesg_num: 0; chunk_offset: 328); adding dummy dev data...
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packa

In [ ]:
#подключаем пакеты для обработки геометрии
from pyproj import CRS
import geopandas as gpd
from shapely.geometry import LineString

In [ ]:
#получаем границы экстента и всей геометрии и центр
line = gpd.GeoDataFrame(crs = CRS('EPSG:4326'), geometry=[LineString(zip(record_df.longitude, record_df.latitude))])
line_centroid = line.centroid
line_bounds = line.bounds

<ipython-input-30-c4d62c76c52f>:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  line_centroid = line.centroid


In [ ]:
#показываем на карте
import folium
m = folium.Map([line_centroid.y, line_centroid.x], tiles='cartodb positron')
folium.GeoJson(line).add_to(m)
folium.FitBounds([[line_bounds.miny[0], line_bounds.minx[0]], [line_bounds.maxy[0],line_bounds.maxx[0]]]).add_to(m)
m

In [ ]:
#строим график
import altair as alt
alt.Chart(record_df).mark_point().encode(
    y='cadence',
    x='distance',
    color='heart_rate',
).interactive()

alt.Chart(...)